## Normal Training with Pytorch

In [1]:
#hide
import torch
from tqdm import tqdm
from torchvision import datasets, transforms

In [ ]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet101', pretrained=False)
model.fc = torch.nn.Linear(2048, 10)
model = model.cuda()

In [3]:
#hide
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False,
                           download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
#hide
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

In [7]:
for epoch in range(1,6):
    model.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data, target in tepoch:
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            tepoch.set_postfix(loss=loss.item())


100%|██████████| 782/782 [00:55<00:00, 14.12batch/s, loss=1.42]


### Using Torch amp

In [5]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet101', pretrained=False)
model.fc = torch.nn.Linear(2048, 10)
model = model.cuda()

Using cache found in /home/venom/.cache/torch/hub/pytorch_vision_v0.6.0


In [6]:
scaler = torch.cuda.amp.GradScaler() # Gradient scaler for amp (Mixed Precision)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

In [7]:
for epoch in range(1, 6):
    model.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data, target in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            with torch.cuda.amp.autocast(): # Automatic Mixed Precision
                output = model(data)
                loss = criterion(output, target)
            scaler.scale(loss).backward() # Scale the loss
            scaler.step(optimizer) # Unscales the gradients of optimizer's assigned params in-place
            scaler.update() # Updates the scale for next iteration
            tepoch.set_postfix(loss=loss.item())


Epoch 5: 100%|██████████| 782/782 [00:46<00:00, 16.79batch/s, loss=1.2] 


There is approx ~20% speedup on this simple model.